In [1]:
import os
os.chdir('../')
%pwd

'e:\\Deep Learning\\pytorch\\facial_emotion_prediction'

In [2]:
train_path = 'artifacts/data_preparation/train'
val_path = "artifacts/data_preparation/val"

In [3]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import mlflow
import mlflow.pytorch
import matplotlib.pyplot as plt

In [4]:
def define_transforms():
    transformer = transforms.Compose([
        transforms.Resize((48, 48)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),  # 0-250 to 0-1 numpy to tensors
        transforms.Normalize([0.5, 0.5, 0.5],
                             [0.5, 0.5, 0.5])
    ])
    return transformer

In [5]:
def data_loader(train_path, test_path, transformer):
    train_loader = DataLoader(
        torchvision.datasets.ImageFolder(train_path, transform=transformer),
        batch_size=16, shuffle=True
    )
    test_loader = DataLoader(
        torchvision.datasets.ImageFolder(test_path, transform=transformer),
        batch_size=16, shuffle=True
    )
    return train_loader, test_loader

In [6]:
def class_name(train_path):
    root = pathlib.Path(train_path)
    classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
    return classes

In [7]:
def train_test_count(train_path, test_path):
    train_count = len(glob.glob(train_path + '/**/*.jpg'))
    test_count = len(glob.glob(test_path + '/**/*.jpg'))
    return train_count, test_count

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [9]:
transformer = define_transforms()
train_loader, test_loader = data_loader(train_path, val_path, transformer)
classes = class_name(train_path)
print(classes)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [10]:
train_count, val_count = train_test_count(train_path, val_path)

In [11]:
print(train_count, ",", val_count)

25117 , 7178


In [12]:
from PIL import Image
img = Image.open('artifacts/data_preparation/train/disgust/416.jpg')
img_channels = len(img.getbands())  
print("Number of channels:", img_channels)

Number of channels: 1


In [13]:
import torch
import torch.nn as nn
from torchsummary import summary

def print_model_summary(model, input_size):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    summary(model, input_size, device=str(device))

def complex_model():
    input_channels = 3  # Update to 3 for RGB images
    img_size = [48, 48]

    cnn = nn.Sequential(
        # Convolutional Layer 1
        nn.Conv2d(input_channels, 32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        
        # Convolutional Layer 2
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        
        # Convolutional Layer 3
        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Additional Convolutional Layer 4
        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Additional Convolutional Layer 5
        nn.Conv2d(256, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        
        # Flatten and Fully Connected Layers
        nn.Flatten(),
        nn.Linear(512 * (img_size[0] // 32) * (img_size[1] // 32), 1024),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 7)  # Output layer adjusted for 7 classes
    )

    print_model_summary(cnn, (input_channels, *img_size))
    return cnn

# Create the complex model
model = complex_model()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 48, 48]             896
       BatchNorm2d-2           [-1, 32, 48, 48]              64
              ReLU-3           [-1, 32, 48, 48]               0
         MaxPool2d-4           [-1, 32, 24, 24]               0
            Conv2d-5           [-1, 64, 24, 24]          18,496
       BatchNorm2d-6           [-1, 64, 24, 24]             128
              ReLU-7           [-1, 64, 24, 24]               0
         MaxPool2d-8           [-1, 64, 12, 12]               0
            Conv2d-9          [-1, 128, 12, 12]          73,856
      BatchNorm2d-10          [-1, 128, 12, 12]             256
             ReLU-11          [-1, 128, 12, 12]               0
        MaxPool2d-12            [-1, 128, 6, 6]               0
           Conv2d-13            [-1, 256, 6, 6]         295,168
      BatchNorm2d-14            [-1, 25

In [14]:
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()
num_epochs = 1

for epoch in range(num_epochs):
    print("Epoch:", epoch)
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))

    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count

    model.eval()

    test_accuracy = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()

            outputs = model(images)
            _, prediction = torch.max(outputs.data, 1)
            test_accuracy += int(torch.sum(prediction == labels.data))

    test_accuracy = test_accuracy / val_count

    print(f'Epoch: {epoch} Train Loss: {train_loss:.4f} Train Accuracy: {train_accuracy:.4f} Test Accuracy: {test_accuracy:.4f}')

Epoch: 0
Epoch: 0 Train Loss: 1.5813 Train Accuracy: 0.3724 Test Accuracy: 0.4579
